20 iteration time check - Done  
AMP and OCP - Done  
collate_fn - Done

In [1]:
MAX_COUNT_ITERATIONS = 10
# MAX_COUNT_ITERATIONS = 1

In [2]:
#uv  (to make pip fast ) - Done
#ref : https://medium.com/bitgrit-data-science-publication/forget-pip-install-use-this-instead-754863c58f1e
# !pip install uv
# !uv venv
# !source .venv/bin/activate

# !uv pip install tokenizers
# !uv pip install torchtext
# !uv pip install pytorch_lightning
# !uv pip install datasets
# !uv pip install tensorboard

#!pip install torchmetrics
#!pip show torchtext

In [3]:
# !pip install --upgrade pytorch
# !pip install --upgrade torchmetrics

In [4]:
!pip install tokenizers
!pip install torchtext
!pip install pytorch_lightning
!pip install datasets
!pip install tensorboard

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [5]:
!git clone https://github.com/rajayourfriend/EraV2/
!cp EraV2/S18/* .

Cloning into 'EraV2'...
remote: Enumerating objects: 829, done.
remote: Counting objects: 100% (210/210), done.
remote: Compressing objects: 100% (144/144), done.
remote: Total 829 (delta 134), reused 142 (delta 66), pack-reused 619
Receiving objects: 100% (829/829), 195.08 MiB | 32.04 MiB/s, done.
Resolving deltas: 100% (306/306), done.
Updating files: 100% (348/348), done.


In [6]:
from config_file import get_config, get_weights_file_path
from train import get_model, get_ds, run_validation
import torch

config = get_config()
config["batch_size"] = 16
config["preload"] = None
# config["num_epochs"] = 1 # Raja ToDo : Number of epochs = 1. Edit num_epochs, for complete run

torch.cuda.amp.autocast(enabled=True)

print("epochs = 1. Edit num_epochs, for complete run")


/usr/local/lib/python3.10/dist-packages/torchtext/datasets/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


epochs = 1. Edit num_epochs, for complete run


In [7]:
!pip install lion_pytorch
from lion_pytorch import Lion

In [ ]:

import warnings
from tqdm import tqdm
import os
from pathlib import Path

import torchmetrics
from torch.utils.tensorboard import SummaryWriter
import time

import torch.nn as nn


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device : {device}")
print("Using device2 :", device)

Path(config["model_folder"]).mkdir(parents=True, exist_ok=True)
train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)

model = get_model(config, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)

#Tensorboard
writer = SummaryWriter(config["experiment_name"])

#Adam is used to train each feature with a different learning rate.
#If some feature is appearing less, adam takes care of it
# optimizer = torch.optim.Adam(model.parameters(), lr = config["lr"], eps = 1e-9)

optimizer = Lion(model.parameters(), lr = 1e-5, weight_decay = 1e-2)

# OCP
MAX_LR = 10**-3
STEPS_PER_EPOCH = len(train_dataloader)
EPOCHS = 18
# Raja ToDo : Number of epochs = 1. Edit num_epochs, for complete run
# EPOCHS = 1
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer,
                                                max_lr=MAX_LR,
                                                steps_per_epoch=STEPS_PER_EPOCH,
                                                epochs=EPOCHS,
                                                pct_start=int(0.3*EPOCHS)/EPOCHS if EPOCHS != 1 else 0.5,
                                                div_factor=100,
                                                three_phase=False,
                                                final_div_factor=100,
                                                anneal_strategy="linear"
                                                )


initial_epoch = 0
global_step = 0

loss_fn = nn.CrossEntropyLoss(ignore_index = tokenizer_src.token_to_id("[PAD]"), label_smoothing=0.1).to(device)

counter = 0
scaler = torch.cuda.amp.GradScaler()
lr = [0.0]
start = time.time()
for epoch in range(initial_epoch, EPOCHS):
  torch.cuda.empty_cache()
  model.train()
  batch_iterator = tqdm(train_dataloader, desc=f"Processing Epoch {epoch:02d}")
  for batch in batch_iterator:
    optimizer.zero_grad(set_to_none=True)
    encoder_input = batch['encoder_input'].to(device)
    decoder_input = batch['decoder_input'].to(device)
    encoder_mask = batch['encoder_mask'].to(device)
    decoder_mask = batch['decoder_mask'].to(device)

    with torch.autocast(device_type='cuda', dtype=torch.float32):
      encoder_output = model.encode(encoder_input, encoder_mask)
      decoder_output = model.decode(encoder_output, encoder_mask, decoder_input, decoder_mask)
      proj_output = model.project(decoder_output)

      label = batch['label'].to(device)

      loss = loss_fn(proj_output.view(-1, tokenizer_tgt.get_vocab_size()), label.view(-1))
    batch_iterator.set_postfix({"loss": f"{loss.item():6.3f}", " lr": f"{lr[-1]}"})

    writer.add_scalar('train loss', loss.item(), global_step)
    writer.flush()

    scaler.scale(loss).backward()
    scale = scaler.get_scale()
    scaler.step(optimizer)
    scaler.update()
    skip_lr_sched = (scale > scaler.get_scale())
    if not skip_lr_sched:
      scheduler.step()
    lr.append(scheduler.get_last_lr())

    global_step += 1



    # run_validation(model, val_dataloader, tokenizer_src, tokenizer_tgt, config['seq_len'], device, lambda msg: batch_iterator.write(msg), global_step)

    model_filename = get_weights_file_path(config, f"{epoch:02d}")
    torch.save({
      'epoch': epoch,
      'model_state_dict': model.state_dict(),
      'optimizer_state_dict': optimizer.state_dict(),
      'global_step': global_step
    }, model_filename)

    counter += 1
    # print("At the end of an iteration.")
    #if(counter == MAX_COUNT_ITERATIONS):
        #print("Breaking here at the end of %   iterations. Remove break, for complete run" % counter)
        #break # Raja ToDo : Delete later

  #if(counter == MAX_COUNT_ITERATIONS):
    #print("Breaking here at the end of %   iterations from outer while loop. Remove break, for complete run" % counter)
    #break # Raja ToDo : Delete later

end = time.time()
print("time taken for the iterations = %s seconds" % (end - start))

Using device : cuda
Using device2 : cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/32332 [00:00<?, ? examples/s]

Max length of the source sentence : 309
Max length of the source target : 274


Processing Epoch 00:  65%|██████▍   | 1176/1819 [1:46:04<1:05:19,  6.10s/it, loss=6.588,  lr=[0.00013802287222344404]]

For 10 iterations in an epoch, it took time = 7.615 seconds  
For 20 iterations in an epoch, it took time = 15.26 seconds

In [ ]:
print(MAX_COUNT_ITERATIONS)